In [1]:
pip install gcvit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for typing: filename=typing-3.7.4.3-py3-none-any.whl size=26324 sha256=b65f1e568a8c95b7638c10da5a3109031f281ed389d9ec91299fe9a660c45263
  Stored in directory: /Users/saqlainnaseemkalokhe/Library/Caches/pip/wheels/7c/d0/9e/1f26ebb66d9e1732e4098bc5a6c2d91f6c9a529838f0284890
Successfully built typing
Note: you may need to restart the kernel to use updated packages.


In [4]:
import tensorflow as tf
import gradio as gr
import gcvit
from gcvit.utils import get_gradcam_model, get_gradcam_prediction


brain_tumor_model_path = 'brain_tumor' 
brain_tumor_model = tf.keras.models.load_model(brain_tumor_model_path)

def predict_fn(image, model_choice):
    if model_choice.startswith('GCViT'):
        model = getattr(gcvit, model_choice)(pretrain=True)
        gradcam_model = get_gradcam_model(model)
        preds, overlay = get_gradcam_prediction(image, gradcam_model, cmap='jet', alpha=0.4, pred_index=None)
        preds = {x[1]: float(x[2]) for x in preds}
        return [preds, overlay]
    elif model_choice == 'Brain Tumor Model':
        # Assuming the image needs preprocessing similar to your earlier example
        class_name=['glioma','meningioma','notumor','pituitary']
        image = tf.image.resize(image, [256, 256])
        image = tf.image.rgb_to_grayscale(image)
        image = tf.expand_dims(image, 0)  # Add batch dimension
        prediction = brain_tumor_model.predict(image)
        preds = {class_name[i]: float(prediction[0][i]) for i in range(len(class_name))}
        return [preds, None]  # No GradCAM for brain tumor model

demo = gr.Interface(
    fn=predict_fn,
    inputs=[
        gr.components.Image(label="Input Image"),
        gr.Radio(['GCViTXXTiny', 'GCViTXTiny', 'GCViTTiny', 'GCViTSmall', 'GCViTBase', 'GCViTLarge', 'Brain Tumor Model'],
                 value='GCViTXXTiny', label='Model Name')
    ],
    outputs=[
        gr.components.Label(label="Prediction"),
        gr.components.Image(label="GradCAM or Overlay Image")
    ],
    title="Multi-Domain Vision Transformers",
    description="Choose a model to perform image classification using ViT with ImageNet pretrain weights or detect brain tumors.",
    examples=[
        ["balloon.png", 'GCViTXXTiny'],
        ["chel.png", 'GCViTXXTiny'],
        ["car.webp", 'GCViTXXTiny'],
        
        
    ],
)
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7863


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/analytics.py:98: UserWarning: unable to parse version details from package URL.
  warnings.warn("unable to parse version details from package URL.")
Exception in thread Thread-19 (_do_normal_analytics_request):
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/httpx/_transports/default.py", line 69, in map_httpcore_exceptions
    yield
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/httpx/_transports/default.py", line 233, in handle_request
    resp = self._pool.handle_request(req)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/httpcore/_sync/connection_pool.py", line 216, in handle_request
    raise exc from None
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/httpcore/_sync/connection_pool.py", line 196, in 


Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


1/1 [==============================] - 0s 149ms/step
